# Useful links

- [ ] https://cran.r-project.org/web/packages/afex/vignettes/introduction-mixed-models.pdf
- [ ] https://cran.r-project.org/web/packages/afex/afex.pdf
- [ ] https://cran.r-project.org/web/packages/afex/index.html
- [ ] https://cran.r-project.org/web/packages/afex/vignettes/afex_analysing_accuracy_data.html
- [ ] https://cran.r-project.org/web/packages/afex/vignettes/afex_anova_example.html
- [ ] https://cran.r-project.org/web/packages/afex/vignettes/afex_mixed_example.html
- [ ] https://cran.r-project.org/web/packages/emmeans/vignettes/comparisons.html
- [ ] https://cran.r-project.org/web/packages/emmeans/vignettes/messy-data.html
- [ ] https://cran.r-project.org/web/packages/emmeans/vignettes/interactions.html
- [ ] https://cran.r-project.org/web/packages/emmeans/vignettes/basics.html
- [ ] https://cran.r-project.org/web/packages/afex/vignettes/afex_mixed_example.html

- [ ] https://osf.io/nfwx7/

- [ ] https://stats.stackexchange.com/questions/13166/rs-lmer-cheat-sheet

- [ ] https://psu-psychology.github.io/r-bootcamp-2018/index.html
- [ ] http://tysonbarrett.com/EDUC-6600/Resources/Example_Ch15_rmANOVA.html
- [ ] http://singmann.org/anova-in-r-afex-may-be-the-solution-you-are-looking-for/
- [ ] https://ademos.people.uic.edu/Chapter21.html
- [ ] https://www.psychologie.uni-heidelberg.de/ae/meth/team/mertens/blog/anova_in_r_made_easy.nb.html
- [ ] http://tysonbarrett.com/EDUC-6600/Resources/Example_Ch15_rmANOVA.html

# Setup

## Imports

In [1]:
library("dplyr")
library("tidyr")
library("afex")     # provides mixed() and attaches lme4 automatically.
library("emmeans")  # follow-up tests.
library("multcomp") # advanced control for multiple testing/Type 1 errors.
library("ggplot2")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: lme4

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Registered S3 methods overwritten by 'car':
  method                          from
  influence.merMod                lme4
  cooks.distance.influence.merMod lme4
  dfbeta.influence.merMod         lme4
  dfbetas.influence.merMod        lme4

************
Welcome to afex. For support visit: http://afex.singmann.science/

- Functions for ANOVAs: aov_car(), aov_ez(), and aov_4()
- Methods for calculating p-values with mixed(): 'KR', 'S', 'LRT', and 'PB'
- 'afex_aov' and 'mixed' objects can be passed to emmeans() for follow-up tests
- NEWS: library('emmeans') now needs to be called explicitly!
- Get and set global pac

In [2]:
library("car")
require("lattice")
require("ez")

Loading required package: carData


Attaching package: ‘car’


The following object is masked from ‘package:dplyr’:

    recode


Loading required package: lattice

Loading required package: ez



In [3]:
library("cowplot")
library("ggbeeswarm")

library("nortest")

In [4]:
# library("apa")

In [5]:
library("effectsize")

## Settings

In [6]:
theme_set(theme_bw(base_size = 15) +
            theme(legend.position="bottom",
                  panel.grid.major.x = element_blank()))

In [7]:
options(width=120)

## Session info

In [8]:
sessionInfo()

R version 3.6.3 (2020-02-29)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 18.04.5 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas/libblas.so.3
LAPACK: /usr/lib/x86_64-linux-gnu/libopenblasp-r0.2.20.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C               LC_TIME=en_IL.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_IL.UTF-8    LC_MESSAGES=en_US.UTF-8    LC_PAPER=en_IL.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C             LC_MEASUREMENT=en_IL.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] effectsize_0.3.3 nortest_1.0-4    ggbeeswarm_0.6.0 cowplot_1.1.0    ez_4.4-0         lattice_0.20-41 
 [7] car_3.0-10       carData_3.0-4    ggplot2_3.3.2    multcomp_1.4-14  TH.data_1.0-10   MASS_7.3-53     
[13] survival_3.2-7   mvtnorm_1.1-1    emmeans_1.5.1    afex_0.28-0    

## Create output directory

In [9]:
dir.create("data/t1007",showWarnings = FALSE,recursive = TRUE)

## Handy functions

### Function for logging information BEFORE execution

In [10]:
logging0 <- function(ifName,infoStr="",data){
    t0 <- Sys.time()
    s0 <- gsub("\\s","_",t0)
    s1 <- gsub("^.","",tempfile("","",""))
    s2 <- gsub("/t1006/","/t1007/",ifName)
    ofName <- paste(paste(s2,"",s0,"",s1,infoStr,"_nrow",nrow(data),sep="_"),".RData",sep="")
    lfName = paste(ofName ,".LOG.txt" ,sep="" ,collapse = "")
    log_msg <- toString(paste(
        paste("logs0" ,toString(lfName)     ,sep=": " ,collapse = "")
      , paste("time0" ,toString(Sys.time()) ,sep=": " ,collapse = "")
      , paste("nrows" ,toString(nrow(data))  ,sep=": " ,collapse = "")
      , paste("ncols" ,toString(ncol(data))  ,sep=": " ,collapse = "")
      , paste("vList" ,toString(ls())       ,sep=": " ,collapse = "")
      , paste("oSave" ,toString(ofName)     ,sep=": " ,collapse = "")        
      , "\n"
      , sep="\n"))
    cat(log_msg)
    write(paste(log_msg) ,file=lfName ,append=TRUE)
    return(list("ofName"=ofName,"lfName"=lfName,"t0"=t0))
}

### Function for logging information AFTER execution

In [11]:
logging1 <- function(lfName,t0){
    t1 <- Sys.time()
    d1 <- t1-t0
    log_msg <- toString(paste(
        paste("time1" ,toString(Sys.time()) ,sep=": " ,collapse = "")
      , paste("diff1" ,capture.output(d1)   ,sep=": " ,collapse = "")
      , "\n"
      , sep="\n"))
    cat(log_msg)
    write(paste(log_msg) ,file=lfName ,append=TRUE)
    return(list("d1" = d1))
}

### Function for displaying results in APA 6

In [12]:
library(xtable)
library(IRdisplay)
library(repr)
require("papaja")
require("tinytex")
apaMd <- function(apaObj){
    apaStr <- ""
    for (ii in names(apaObj)){
        apaStr <- paste(apaStr, paste("\n","### ", ii, "\n" ,sep="" ,collapse = "")  ,sep="" ,collapse = "")
        if(!(ii == "table")){ 
            for (jj in names(apaObj[[ii]])){
                apaStr <- paste(apaStr, paste("- ", ii," for `", jj, "` is ", "\n" ,sep="" ,collapse = "")  ,sep="\n" ,collapse = " ")
                apaStr <- paste(apaStr, paste("  - ", apaObj[[ii]][[jj]], "\n" ,sep="" ,collapse = "")  ,sep="\n" ,collapse = " ")
                }}
        else {
            # apaStr <- paste(apaStr, "\n\n", paste(apa_table(apaObj$table))  ,sep="\n" ,collapse = " ")
            apaStr <- paste(apaStr, paste("- support for will arrive some day", "\n" ,sep="" ,collapse = "")  ,sep="\n" ,collapse = " ")
            apaStr <- paste(apaStr, paste("- for now please use apa_table function", "\n" ,sep="" ,collapse = "")  ,sep="\n" ,collapse = " ")

        }}
    return(apaStr)
}


Attaching package: ‘IRdisplay’


The following object is masked from ‘package:xtable’:

    display


Loading required package: papaja

Loading required package: tinytex



# Load data

Depending on how much data you want to include select a file to be loaded (either by executing relevant cell or by clicking ↑ or ↓ above to move cells up or down, the last executed cell containing `ifName` defines the file to be loaded).

In [13]:
ifName="data/t1006/bigDF__df2__001_unstacked.RData" # 752554 obs

In [14]:
load(file=ifName)
df3 <- df2

In [15]:
df2 <- subset(df2,wordCnt>4) # ~649262 obs

In [17]:
df2 = subset(df2, select = -c(nn01Cnt,ag02Cnt,nn01Prc,ag02Prc) )

In [16]:
df2$LBL <- paste(df2$CAMP,"_",df2$TYPE, sep="")

In [18]:
df2$vb01Prc <- df2$vb01Prc*100

In [19]:
df2$ag01Prc <- df2$ag01Prc*100

# Inspect data

## Quick look at data

In [20]:
str(df2)

'data.frame':	681652 obs. of  17 variables:
 $ CAMP       : Factor w/ 2 levels "metoo","sexstrike": 1 1 1 1 1 1 1 1 1 1 ...
 $ TYPE       : Factor w/ 2 levels "ctrl","orig": 1 1 1 1 1 1 1 1 1 1 ...
 $ wordCnt    : num  11 7 11 27 30 26 8 12 12 23 ...
 $ vb01Cnt    : num  0 0 0 2 0 0 1 0 1 0 ...
 $ ag01Cnt    : num  0 0 0 0 1 1 0 0 1 2 ...
 $ vb01Prc    : num  0 0 0 7.41 0 ...
 $ ag01Prc    : num  0 0 0 0 3.33 ...
 $ ID         : chr  "919714354467045376" "919714354479722496" "919714354500653056" "919714354542600192" ...
 $ tweet_url  : chr  "/IamJonesy3/status/919714354467045376" "/CCmee_/status/919714354479722496" "/SarahJaneVlogs/status/919714354500653057" "/queentyreen/status/919714354542600195" ...
 $ user_id    : chr  "588400632" "242582835" "29298715" "3290716455" ...
 $ is_replied : logi  TRUE FALSE FALSE TRUE FALSE TRUE ...
 $ is_reply_to: logi  TRUE FALSE FALSE TRUE FALSE FALSE ...
 $ likes      : num  1 0 1 0 0 38 0 0 1 1 ...
 $ replies    : num  1 0 0 1 0 5 0 0 0 0 ...
 $ re

## Summary

In [21]:
summary(df2)

        CAMP          TYPE           wordCnt          vb01Cnt           ag01Cnt           vb01Prc       
 metoo    :613635   ctrl:353731   Min.   :  5.00   Min.   : 0.0000   Min.   : 0.0000   Min.   :  0.000  
 sexstrike: 68017   orig:327921   1st Qu.: 11.00   1st Qu.: 0.0000   1st Qu.: 0.0000   1st Qu.:  0.000  
                                  Median : 18.00   Median : 1.0000   Median : 0.0000   Median :  2.439  
                                  Mean   : 18.43   Mean   : 0.8063   Mean   : 0.6863   Mean   :  4.413  
                                  3rd Qu.: 24.00   3rd Qu.: 1.0000   3rd Qu.: 1.0000   3rd Qu.:  7.547  
                                  Max.   :106.00   Max.   :17.0000   Max.   :21.0000   Max.   :100.000  
    ag01Prc             ID             tweet_url           user_id          is_replied      is_reply_to    
 Min.   :  0.000   Length:681652      Length:681652      Length:681652      Mode :logical   Mode :logical  
 1st Qu.:  0.000   Class :character   Class :char

In [22]:
ofName="data/t1007/bigDF__df2_unstacked__clean.RData" # 752554 obs
save(df2,file=ofName)

# ANOVA (`aov_4`)

| name         | meta   | descr                       |
|--------------|--------|-----------------------------|
| `ID`         | `u1`   | observation unit (tweet_id) |
| `vb01Prc`    | `d1`   | outcome (dependent)         |
| `ag01Prc`    | `d2`   | outcome (dependent)         |
| `CAMP`       | `b1`   | between tweet               |
| `TYPE`       | `b2`   | between tweet               |

In [23]:
OBSERVED <- c("CAMP","TYPE")

In [24]:
OBSERVED <- NULL

## AOV for Verbs

In [25]:
FORMULA <- vb01Prc~CAMP*TYPE+(1|ID)

log0 <- logging0(ifName,infoStr="_aov_4_a1vb01Prc",data=df2)
a1vb01Prc <- aov_4(
    formula       <- FORMULA,
    data          <- df2,
    observed      <- OBSERVED,
)
save(a1vb01Prc,file=log0$ofName)
log1 <- logging1(log0$lfName,log0$t0)

ofName="data/t1007/bigDF__df2_unstacked__aov_4_a1vb01Prc__nrow_681652.RData" # 752554 obs
save(a1vb01Prc,file=ofName)

logs0: data/t1007/bigDF__df2__001_unstacked.RData__2020-10-18_04:37:07__7fd827dc9486__aov_4_a1vb01Prc__nrow_681652.RData.LOG.txt
time0: 2020-10-18 04:37:07
nrows: 681652
ncols: 17
vList: data, ifName, infoStr, lfName, ofName, s0, s1, s2, t0
oSave: data/t1007/bigDF__df2__001_unstacked.RData__2020-10-18_04:37:07__7fd827dc9486__aov_4_a1vb01Prc__nrow_681652.RData



Contrasts set to contr.sum for the following variables: CAMP, TYPE



time1: 2020-10-18 04:38:01
diff1: Time difference of 54.17546 secs



## AOV for Agency Dictionary

In [26]:
FORMULA <- ag01Prc~CAMP*TYPE+(1|ID)

log0 <- logging0(ifName,infoStr="_aov_4_a2ag01Prc",data=df2)
a2ag01Prc <- aov_4(
    formula       <- FORMULA,
    data          <- df2,
    observed      <- OBSERVED,
)
save(a2ag01Prc,file=log0$ofName)
log1 <- logging1(log0$lfName,log0$t0)

ofName="data/t1007/bigDF__df2_unstacked__aov_4_a2ag01Prc__nrow_681652.RData" # 752554 obs
save(a2ag01Prc,file=ofName)

logs0: data/t1007/bigDF__df2__001_unstacked.RData__2020-10-18_04:38:43__7fd83cd061bd__aov_4_a2ag01Prc__nrow_681652.RData.LOG.txt
time0: 2020-10-18 04:38:43
nrows: 681652
ncols: 17
vList: data, ifName, infoStr, lfName, ofName, s0, s1, s2, t0
oSave: data/t1007/bigDF__df2__001_unstacked.RData__2020-10-18_04:38:43__7fd83cd061bd__aov_4_a2ag01Prc__nrow_681652.RData



Contrasts set to contr.sum for the following variables: CAMP, TYPE



time1: 2020-10-18 04:39:35
diff1: Time difference of 51.60162 secs



# Quick look at AOV results

In [27]:
summary(a1vb01Prc)

,num Df,den Df,MSE,F,ges,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CAMP,1,681648,33.73904,690.32244045,1.011701e-03,4.531552e-152
TYPE,1,681648,33.73904,0.02922063,4.286762e-08,8.642707e-01
CAMP:TYPE,1,681648,33.73904,142.14341243,2.084856e-04,9.115624e-33


In [28]:
summary(a2ag01Prc)

,num Df,den Df,MSE,F,ges,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CAMP,1,681648,36.20482,1102.265888,0.0016144496,1.659872e-241
TYPE,1,681648,36.20482,5.538231,0.0000081247,1.860567e-02
CAMP:TYPE,1,681648,36.20482,506.682508,0.0007427678,3.673188e-112


In [ ]:
#                      ___           ___                    ___           ___                   
#                     /\  \         /\__\                  /\__\         /\  \         _____    
#        ___          \:\  \       /:/ _/_                /:/ _/_        \:\  \       /::\  \   
#       /\__\          \:\  \     /:/ /\__\              /:/ /\__\        \:\  \     /:/\:\  \  
#      /:/  /      ___ /::\  \   /:/ /:/ _/_            /:/ /:/ _/_   _____\:\  \   /:/  \:\__\ 
#     /:/__/      /\  /:/\:\__\ /:/_/:/ /\__\          /:/_/:/ /\__\ /::::::::\__\ /:/__/ \:|__|
#    /::\  \      \:\/:/  \/__/ \:\/:/ /:/  /          \:\/:/ /:/  / \:\~~\~~\/__/ \:\  \ /:/  /
#   /:/\:\  \      \::/__/       \::/_/:/  /            \::/_/:/  /   \:\  \        \:\  /:/  / 
#   \/__\:\  \      \:\  \        \:\/:/  /              \:\/:/  /     \:\  \        \:\/:/  /  
#        \:\__\      \:\__\        \::/  /                \::/  /       \:\__\        \::/  /   
#         \/__/       \/__/         \/__/                  \/__/         \/__/         \/__/    
#  




The following is redundant as any further results inspections are done in the separate file

## Summarize data by `CAMP` ⨯ `TYPE`

In [ ]:
smr0 <- df2 %>%
    group_by(CAMP,TYPE) %>%
    summarise(
        N = n(),
        wcM = mean(wordCnt),
        wcSD = sd(wordCnt),
        wcSE = wcSD / sqrt(N),
        vbM = mean(vb01Prc),
        vbSD = sd(vb01Prc),
        vbSE = vbSD / sqrt(N),
        agM = mean(ag01Prc),
        agSD = sd(ag01Prc),
        agSE = agSD / sqrt(N),
        .groups = "drop"
    ) 
smr0

## Summarize data by `CAMP`

In [ ]:
smr0 <- df2 %>%
    group_by(CAMP) %>%
    summarise(
        N = n(),
        wcM = mean(wordCnt),
        wcSD = sd(wordCnt),
        wcSE = wcSD / sqrt(N),
        vbM = mean(vb01Prc),
        vbSD = sd(vb01Prc),
        vbSE = vbSD / sqrt(N),
        agM = mean(ag01Prc),
        agSD = sd(ag01Prc),
        agSE = agSD / sqrt(N),
        .groups = "drop"
    ) 
smr0

## Summarize by type `TYPE`

In [ ]:
smr0 <- df2 %>%
    group_by(TYPE) %>%
    summarise(
        N = n(),
        wcM = mean(wordCnt),
        wcSD = sd(wordCnt),
        wcSE = wcSD / sqrt(N),
        vbM = mean(vb01Prc),
        vbSD = sd(vb01Prc),
        vbSE = vbSD / sqrt(N),
        agM = mean(ag01Prc),
        agSD = sd(ag01Prc),
        agSE = agSD / sqrt(N),
        .groups = "drop"
    ) 
smr0

## Generate APA-like LaTeX for AOV

In [ ]:
apa0vb <- apa_print(a1vb01Prc)
apa0ag <- apa_print(a2ag01Prc)

In [ ]:
apa0vb$table

In [ ]:
display_markdown(apaMd(apa0vb["full_result"]))

In [ ]:
apa0ag$table

In [ ]:
display_markdown(apaMd(apa0ag["full_result"]))

# Confidence intervals

In [ ]:
cohens_d(ag01Prc~CAMP, data = df2)

In [ ]:
cohens_d(ag01Prc~TYPE, data = df2)

# Estimated marginal means (Least-squares means)

## Full model

In [ ]:
a1vb01PrcEm1 <- emmeans(a1vb01Prc, specs<-c("CAMP", "TYPE"))
a2ag01PrcEm1 <- emmeans(a2ag01Prc, specs<-c("CAMP", "TYPE"))

In [ ]:
a1vb01PrcEm1

In [ ]:
a2ag01PrcEm1

In [ ]:
pairs(a1vb01PrcEm1)

In [ ]:
pairs(a2ag01PrcEm1)

In [ ]:
TEST=adjusted("bonferroni")

In [ ]:
TEST=adjusted("free")

In [ ]:
summary(as.glht(pairs(a1vb01PrcEm1)),test=TEST)

In [ ]:
summary(as.glht(pairs(a2ag01PrcEm1)),test=TEST)

## Contrasts

# coef(contrast(a1vb01PrcEm1))

In [ ]:
coef(contrast(a2ag01PrcEm1))

In [ ]:
apa0vbEm1 <- apa_print(a1vb01PrcEm1)
apa0agEm1 <- apa_print(a2ag01PrcEm1)

In [ ]:
apa0vbEm1$table

In [ ]:
apa0agEm1$table

In [ ]:
display_markdown(apaMd(apa0vbEm1["full_result"]))

In [ ]:
display_markdown(apaMd(apa0agEm1["full_result"]))

In [ ]:
apa0vbEm1Pa <- apa_print(pairs(a1vb01PrcEm1))
apa0agEm1Pa <- apa_print(pairs(a2ag01PrcEm1))

In [ ]:
apa0vbEm1Pa$table

In [ ]:
apa0agEm1Pa$table

In [ ]:
display_markdown(apaMd(apa0vbEm1Pa["full_result"]))

In [ ]:
display_markdown(apaMd(apa0agEm1Pa["full_result"]))

# Handy plots

Quote from: https://cran.r-project.org/web/packages/emmeans/vignettes/comparisons.html
> The blue bars are confidence intervals for the EMMs, and the red arrows are for the comparisons among them. If an arrow from one mean overlaps an arrow from another group, the difference is not “significant,” based on the adjust setting (which defaults to "tukey") and the value of alpha (which defaults to 0.05). See the “xplanations” supplement for details on how these are derived.

In [1]:
library(repr)
options(repr.plot.width=8, repr.plot.height=4)
# fig.height = 10
# fig.width = 5
plot(a1vb01PrcEm2, comparisons=TRUE, xlab="% Verb")+
coord_fixed(ratio = 0.004,xlim=c(0.030, 0.055),ylim=NULL,expand=TRUE,clip="on")
#+ 
#  theme(
#      plot.margin = margin(.2,.2,.2,.2, "cm"),
#      plot.background = element_rect(fill = "darkgrey")
#  )

plot(a2ag01PrcEm2, comparisons=TRUE, xlab="% Dict")+
coord_fixed(ratio = 0.004,xlim=c(0.030, 0.055),ylim=NULL,expand=TRUE,clip="on")


ERROR: Error in plot(a1vb01PrcEm2, comparisons = TRUE, xlab = "% Verb"): object 'a1vb01PrcEm2' not found


## Effect of TYPE (ctrl, orig) conditional on CAMP (#sexstrike, #metoo)

In [ ]:
a1vb01PrcEm2 <- emmeans(a1vb01Prc, specs<-c("TYPE"), by<-"CAMP")
a2ag01PrcEm2 <- emmeans(a2ag01Prc, specs<-c("TYPE"), by<-"CAMP")

In [ ]:
a1vb01PrcEm2

In [ ]:
a2ag01PrcEm2

In [ ]:
pairs(a1vb01PrcEm2)

In [ ]:
pairs(a2ag01PrcEm2)

In [ ]:
summary(as.glht(pairs(a1vb01PrcEm2)),test=TEST)

In [ ]:
summary(as.glht(pairs(a2ag01PrcEm2)),test=TEST)

# Helpers

In [ ]:
?mixed

In [ ]:
?scale

In [ ]:
?lmer

In [ ]:
?aov_4

In [ ]:
?emmeans

In [ ]:
?coord_fixed

# Extras

In [ ]:
df2$retweets <- as.numeric(df2$retweets) # TODO FIXME This should be fixed elsewhere!

In [ ]:
df2$ReTw <- cut(df2$retweets,
                breaks=c(0, 5, 25, 125, Inf), 
                right  = FALSE,
              # labels=c("low","med","high","viral")
               )

str(df2)

# Playground

In [ ]:
plot_grid(
  afex_plot(
      a1vb01Prc, 
      x="TYPE", 
      trace="CAMP",
      error = "between", 
      data_geom = geom_quasirandom, 
      data_alpha = 0.3) + 
    coord_cartesian(ylim = c(0, 1)),
  afex_plot(
      a2ag01Prc, 
      x="TYPE", 
      trace="CAMP",
      error = "between", 
      data_geom = geom_quasirandom, 
      data_alpha = 0.3) +
    coord_cartesian(ylim = c(0, 1))
)

In [ ]:
qqPlot(a1vb01Prc$aov$residuals)

In [ ]:
qqPlot(a2ag01Prc$aov$residuals)

In [ ]:
shapiro.test(a1vb01Prc$aov$residuals)

In [ ]:
shapiro.test(a2ag01Prc$aov$residuals)

In [ ]:
ad.test(a1vb01Prc$aov$residuals)$p.value

In [ ]:
ad.test(a2ag01Prc$aov$residuals)$p.value

In [ ]:
test_levene(a1vb01Prc)

In [ ]:
test_levene(a2ag01Prc)

In [ ]:
library("pwr")

In [ ]:
?pwr.anova.test

In [ ]:
?summary

In [ ]:
convert_ipynb

In [ ]:
?apa_print


In [ ]:
methods("apa_print")

# Sanity checkup for nouns

In [ ]:
ifName="data/t1006/bigDF__df1.RData" # 752554 obs

In [ ]:
load(file=ifName)

In [ ]:
df1 <- subset(df1,wordCnt>4) # ~649262 obs

In [ ]:
df1[ df1$nn01Prc > 1,c('text','twitTokCFD_VB_lst','twitTokCFD_NN_lst','wordCnt','vb01Cnt','nn01Cnt','ag01Cnt','ag02Cnt','vb01Prc','nn01Prc','ag01Prc','ag02Prc')]